# Multi GPU comparison

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import make_moons
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD

assert tf.__version__ == '2.0.0-alpha0'

## Generate fake data

In [3]:
X, y = make_moons(n_samples=1000,
                  noise=0.1,
                  shuffle=True,
                  random_state=0)

## Model generating function

In [4]:
def simple_model():
    model = Sequential([
        Dense(8, input_shape=(2,), activation='tanh'),
        Dense(2, activation='tanh'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(lr=0.1),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

## Tensorflow Distribution Strategy

In [5]:
strategy = tf.distribute.MirroredStrategy()

W0501 12:28:31.880259 140736240931712 cross_device_ops.py:1111] Not all devices in `tf.distribute.Strategy` are visible to TensorFlow.


In [6]:
with strategy.scope():
    model = simple_model()

In [7]:
model.fit(X, y,
          batch_size=32,
          epochs=2,
          shuffle=True)

Epoch 1/2
32/32 [==============================] - 0s 6ms/step - loss: 0.3618 - accuracy: 0.8398
Epoch 2/2
32/32 [==============================] - 0s 2ms/step - loss: 0.2791 - accuracy: 0.8750


## Keras Multi GPU Model

In [8]:
from tensorflow.keras.utils import multi_gpu_model

In [9]:
with tf.device("/cpu:0"):
    model = simple_model()

In [10]:
model = multi_gpu_model(model, 2, cpu_relocation=True)

ValueError: To call `multi_gpu_model` with `gpus=2`, we expect the following devices to be available: ['/cpu:0', '/gpu:0', '/gpu:1']. However this machine only has: ['/cpu:0']. Try reducing `gpus`.

## Exercises

Tensorflow 2.0 includes other distribution strategies beyond the `MirroredStrategy`. Take a look at their documentation [here](https://www.tensorflow.org/alpha/guide/distribute_strategy) and try to use another distribution strategy. Discuss with the person next to you what they serve.